Interactive visualization of standard overcurrent protection curves || Visualização iterativa das curvas padronizadas de proteção de sobrecorrente

Creator: Prof. Pedro Henrique Aquino Barra (pedrobarra@ufu.br)

Version: 1.0.1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (interact, Dropdown, FloatSlider, IntSlider,
                        FloatText, IntText, HBox, VBox, jslink, 
                        interactive_output)

# Curve calculation functions
def ansi_curve(curve_type, time_multiplier, pickup_current):
    """Calculate ANSI overcurrent protection curve parameters"""
    parameters = {
        "I": (0.0086, 0.0185, 0.02),     # Standard Inverse
        "MI": (2.855, 0.0712, 2.0),      # Moderately Inverse
        "EI": (6.407, 0.025, 2.0)        # Extremely Inverse
    }
    A, B, p = parameters[curve_type]
    current = np.logspace(np.log10(pickup_current*1.01), 
                         np.log10(pickup_current*20), 
                         1000)
    trip_time = time_multiplier * (A / ((current/pickup_current)**p - 1) + B)
    return current, trip_time

def iec_curve(curve_type, time_multiplier, pickup_current):
    """Calculate IEC overcurrent protection curve parameters"""
    parameters = {
        "NI": (0.14, 0.02),              # Normal Inverse
        "MI": (13.5, 1),                 # Very Inverse
        "EI": (80, 2),                   # Extremely Inverse
        "LON6": (80, 1)                  # Long Time Inverse
    }
    k, alpha = parameters[curve_type]
    current = np.logspace(np.log10(pickup_current*1.01),
                         np.log10(pickup_current*20),
                         1000)
    trip_time = (k * time_multiplier) / ((current/pickup_current)**alpha - 1)
    return current, trip_time

def plot_curves(standard='Both', time_multiplier=1.0, pickup_current=100):
    """Generate interactive plot of overcurrent curves"""
    plt.figure(figsize=(12, 7))
    
    # Plot ANSI curves if selected
    if standard in ['ANSI', 'Both']:
        for curve in ["I", "MI", "EI"]:
            current, trip_time = ansi_curve(curve, time_multiplier, pickup_current)
            plt.loglog(current, trip_time, '-', linewidth=2, label=f"ANSI {curve}")
    
    # Plot IEC curves if selected
    if standard in ['IEC', 'Both']:
        for curve in ["NI", "MI", "EI", "LON6"]:
            current, trip_time = iec_curve(curve, time_multiplier, pickup_current)
            plt.loglog(current, trip_time, '--', linewidth=2, label=f"IEC {curve}")
    
    # Plot configuration
    plt.xlabel("Secondary Current (A)", fontsize=12)
    plt.ylabel("Trip Time (s)", fontsize=12)
    plt.title(f"Overcurrent Curves (TM={time_multiplier}, Ip={pickup_current}A)", fontsize=14)
    plt.grid(True, which="both", linestyle='--', alpha=0.7)
    plt.legend(loc='upper right', frameon=False)
    plt.xlim(1, 1000)
    plt.ylim(0.01, 1000)
    plt.show()

# Create interactive widgets
standard_widget = Dropdown(
    options=['Both', 'ANSI', 'IEC'],
    value='Both',
    description='Standard:',
    layout={'width': '300px'}
)

# Time Multiplier widgets
tm_slider = FloatSlider(value=0.5, min=0.1, max=2.0, step=0.01, description='TM:')
tm_text = FloatText(value=0.5, description='TM:', layout={'width': '150px'})
jslink((tm_slider, 'value'), (tm_text, 'value'))

# Pickup Current widgets
ip_slider = IntSlider(value=10, min=10, max=200, step=1, description='Ip (A):')
ip_text = IntText(value=10, description='Ip (A):', layout={'width': '150px'})
jslink((ip_slider, 'value'), (ip_text, 'value'))

# Create widget layout
ui = HBox([
    VBox([standard_widget]),
    VBox([
        HBox([tm_slider, tm_text]),
        HBox([ip_slider, ip_text])
    ])
])

# Connect widgets to plot function
out = interactive_output(
    plot_curves,
    {'standard': standard_widget, 
     'time_multiplier': tm_slider, 
     'pickup_current': ip_slider}
)

# Display the interface
display(ui, out)

Output()